In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches

from pickle import dump, load
import xgboost as xgb
from hmmlearn import hmm
import importlib
import sys

sys.path.append('../src/')
import run_model as rm
importlib.reload(rm)

this_path = os.getcwd()
print(this_path)

/home/jemima/Data/scripts_data_results/scripts


# Run Classification Model

In [ ]:
scaler_path = '../model_versions/std_scaler.pkl'
model_path = '../model_versions/final_model.json'

df_train = pd.read_csv("../training_data/training_cleaned_scaled_fsel.csv")
var_cols = [c for c in df_train.columns if c not in ['class']]

for year in np.linspace(2016, 2023, 8).astype(int):
    f_path, outpath = f"../production_data/riverstate/{year}.nc", f"../test_results/riverstate/{year}.nc"
    rm.run_model(f_path, outpath, var_cols, scaler_path, model_path, save_results=True, plot_results=False)

# Hidden Markov Model

In [ ]:
def stack_res(dirpath):
    f_list = sorted([filename for filename in os.listdir(dirpath)])
    da_list = []
    for fname in f_list:
        da_list.append(xr.open_dataarray(os.path.join(dirpath, fname), mask_and_scale=True))
    da_merge = xr.concat(da_list, dim='time') 
    return da_merge

da_res = stack_res("../test_results/riverstate/")

In [ ]:
# Define the state space
states = ['Bare', 'Standard Mangroves', 'Tall Mangroves/Forest']
n_states = len(states)
print('Number of hidden states :', n_states)

# Define the observation space
observations = [0, 1, 2]
n_observations = len(observations)
print('Number of observations  :', n_observations)

# Define the initial state distribution
state_probability = np.array([0.15, 0.4, 0.45])
print("State probability:", state_probability)

# Define the state transition probabilities
transition_probability = np.array([[1.0, 0.0, 0.0],
                                   [0.4, 0.6, 0.0],
                                   [0.1, 0.0, 0.9]])
print("\nTransition probability:\n", transition_probability)

# Define the observation likelihoods
emission_probability = np.array([[1.0, 0.0, 0.0],
                                 [0.3, 0.6, 0.1],
                                 [0.1, 0.1, 0.8]])
print("\nEmission probability:\n", emission_probability)

model = hmm.CategoricalHMM(n_components=n_states)
model.startprob_ = state_probability
model.transmat_ = transition_probability
model.emissionprob_ = emission_probability

In [ ]:
# Apply HMM to data
arr = da_res.to_numpy()
print(arr.shape)

for i in range(arr.shape[2]):
    for j in range(arr.shape[1]):
        if str(arr[0, j, i]) != 'nan':
            try:
                arr[:, j, i] = model.predict(arr[:, j, i].reshape(-1, 1).astype(int))
            except IndexError:
                continue
    print('\r Progress complete (%) - ' + str((i / arr.shape[2]) * 100), end='', flush=True)

In [ ]:
da_hmm = xr.DataArray(arr, coords={
    'time':da_res.time,
    'y':da_res.y.values,
    'x':da_res.x.values
})

da_hmm.to_netcdf("../final_results/hmm_stack.nc")